# Advanced Sklearn Tools and Multi-Class and Imbalanced Class Machine Learning 

<b> Goals: </b>

- Learn how to use an array of advanced tools in sklearn such as Pipelines, GridSearch, and more
- Work on a supervised classification dataset with more than two classes, specifically the famous MNIST digits dataset.
- Work on a supervised classification dataset with imbalanced classes, specifically the credit card fraud dataset.

## Advanced Sklearn tools 

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.metrics import recall_score, roc_auc_score, roc_curve, accuracy_score
from sklearn.preprocessing import Imputer, OneHotEncoder, LabelEncoder, LabelBinarizer

In [ ]:
#Load in titanic data

path = "../../data/titanic.csv"

titanic = pd.read_csv(path)
titanic.head()

### Encoding aka dummy variables with sklearn

**One hot encoding:*** Transforming categorical variables 

In [ ]:
#Assign X and y

X = titanic.drop("Survived", axis = 1)
y = titanic.Survived

In [ ]:
#Make a train test split with the titanic data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 8)

We're going to use LabelEncoder to turn the object values into numbers. Instead turning each unique value into a column a la dummy variables, this tool returns a single column and replaces the objects/strings with a number.

In [ ]:
#Intialize LabelEncoder object



#Use le on the sex column



Turns male and female into a 1s and 0s.

The advantage of using this is that we can use the LabelEncoder object (le) to transform other data.

In [ ]:
#Pass in the Sex column on the testing dataset into the le object




Now let's try this on the Embarked column

In [ ]:

#Intialize LabelEncoder



#Pass Embarked column into le object


#Look at first twenty rows


In [ ]:
# Call .classes_ to see the original object values


In [ ]:
#Transform the embarked class of the testing dataset




In [ ]:
#Look at original X_test.Embarked




How to use the OneHotEncoder object

In [ ]:

#Intialize object


#Fit and transform using the emb_encoded variable




#Look at emb_onehot




In [ ]:
#Transform emb_encoded_test using onehot object




We can also use the LabelBinarizer to do this as well

In [ ]:

#Intialize LabelBinarizer



#Fit and transform on the Embarked column of the training dataset

#Fill nans with unknown



In [ ]:
#Look at the class or column values




In [ ]:
#Transform the testing data using lb




You may be asking yourself "Why use this instead of pd.get_dummies?"

That's because testing data or any other new you want to use may not have the same values in their categorical columns.

In [ ]:
#Create new dataset from X_test where there is no C value in the Embarked column
X_test2 = X_test[X_test.Embarked != "C"]


#Transform the Embarked column from  X_test2 using the LabelBinarizer




This returns a 0 for every value under the C column. Using pd.get_dummies we would have three columns instead of four. This is important because when you fit you model using the training data and then make predictions using the testing data, the model won't work if your testing and training data don't have the same number of columns.

In conclusion:

![e](https://chrisalbon.com/images/machine_learning_flashcards/One-Hot_Encoding_print.png)

## Grid Search and Pipelines

Algorithm that tests every combination of model parameters to find the best one.

Let's use GridSearch to find the best K value for a KNN model and Spotify data

In [ ]:
#Load in spotify data and assign X/y
spotify = pd.read_csv("../../data/spotify_data.csv", index_col=[0])

X = spotify.drop("target", axis = 1)
y = spotify.target

In [ ]:
#Intialize parameter grid

#Range of neighbors to test



#Dictionary of parameter values 
param_grid_knn = {}



In [ ]:
#Intialize Grid

grid_knn = 


#Fit grid on data



In [ ]:
#View scores



In [ ]:
#Whats the best cross validated accuracy score



In [ ]:
#Find the best parameters



This simple technique gives us the best K value.

We can use the best model from grid_knn to make predictions.

In [ ]:
#Input 
x = [[0.2, .15, 0.68, 0.05, 0.328]]

#Make prediction 



In [ ]:
#Proability



In [ ]:


#Range of neighbors to test
depths_range = range(2, 20)

#Dictionary of parameter values 
param_grid_dt = {}



In [ ]:
#Intialize Grid

grid_dt = 


#Fit grid on data



In [ ]:
#Best score for DT model
grid_dt.best_score_

In [ ]:
#Best parameter for DT model
grid_dt.best_params_

In [ ]:
#Make prediction

grid_dt.predict(x)

So far our grids have been one-dimensional, now let's try using multiple dimensions

In [ ]:
#Param grid with test different split criteria as well.
param_grid_dt = {'min_samples_split' : range(2, 30 ,2), "max_depth": depths_range}

param_grid_dt

In [ ]:
#Intialize Grid
grid_dt = 

#Fit grid on data


In [ ]:
#Best parameter

grid_dt.best_params_

In [ ]:
#Best score

grid_dt.best_score_

How many models did the grid search algorithm fit?

In [ ]:
#Add another dimension
param_grid_dt["max_features"] = [1, 2, 3, 4, 5]

In [ ]:
#import time 
from time import time

In [ ]:
#Intialize Grid
grid_dt = GridSearchCV(estimator = DecisionTreeClassifier(), 
                        param_grid = param_grid_dt, cv = 5, scoring = "accuracy")

#Time the code 

t = time()

#Fit grid on data
grid_dt.fit(X, y)

#Print time difference

print (time() - t)

In [ ]:
#Best parameter

grid_dt.best_params_

In [ ]:
#Best score

grid_dt.best_score_

Obviously grid search takes a long time and in some case can cause memory errors. This is where RandomizedSearchCV comes in.

In [ ]:
#Intialize RandomizedSearchCV grid with n_iter = 15
grid_dt = 


#Time the code 

t = time()

#Fit grid on data


#Print time difference

print (time() - t)

Reduced run time by  a whole a lot. But now let's see if we sacrificed performance.

In [ ]:
#Check best score




### **Pipelines**

Let's go back to using the KNN model.

We know that we need to scale our data for the KNN algorithm right?

In [ ]:
#Scale data and fit it a Grid search function it.

#Intialize scalar
# scale = StandardScaler()

#Fit and transform scaler on the data
# Xs = scale.fit_transform(X)

Time to make a pipeline

In [ ]:
#Pass scaler and knn classifier objects into make_pipeline function
pipe = 


In [ ]:
#Create new param_grid
neighbors_range = range(2, 21)
param_grid_knn = {}
param_grid_knn["kneighborsclassifier__n_neighbors"] = neighbors_range
param_grid_knn

In [ ]:
#Pass in pipe into GridSearchCV function, 
grid_knn_pipe = 


#Fit on original versions of data



#Best scores and params
print (grid_knn_pipe.best_score_, grid_knn_pipe.best_params_)

In [ ]:
# Use the cross-validation process using Pipeline
pipe = 


## Multi-class Supervised Learning

So far in our classification lessons we've mainly modeling binary classification datasets aka either-or data. In this class we're going to work through the MNIST digits dataset and learn to work with and interpret models trained on multiple classes—multiple meaning more than two.

Before we get into the MNIST dataset, let's bring back the Iris dataset

In [ ]:
#Load in iris data using seaborn
iris = sb.load_dataset("iris")

iris.head()

The iris dataset is a mulit-class dataset because there are three uniques values in the dependent variable

In [ ]:
#The class of species
iris.species.unique()

Now let's model this data and using a confusion matrix to analyze the results.

In [ ]:
#Step 1. Assign X and y

X = 

y = 

#Step 2. train test split

X_train, X_test, y_train, y_test = 


#Step 3. Fit KNN model with 3 neighbors on training data

knn = 



#Step 4. Make predictions on the X_test using the model

preds = 

#Step 5. Score predictions




Pretty decent model right? Now let's use the confusion matrix to better understand our predictions, particulary the wrong ones.

In [ ]:

#Call pass in y_test and preds into confusion_matrix object

cm = 


cm

Let's turn this into a dataframe

In [ ]:
y.unique()

In [ ]:
#Column and index values for our confusion matrix dataframe
cols = ["pred_setosa", "pred_versicolor", "pred_virginica"]
index = ["actual_setosa", "actual_versicolor", "actual_virginica"]

#Make dataframe out of confusion matrix

cm_df = pd.DataFrame(cm, columns=cols, index=index)

cm_df

The accuracy score told us that our model correctly classified 88% of the testing dataset.

What does the confusion matrix tell us that the accuracy score doesn't? What is the accuracy if we ignore setosa?

In [ ]:
#Non setosa accuracy score



The 13, 15, 16, represent the values that we correctly identified.

The two 3 values represent the values that we incorrectly identified.

<br>

If we want to calculate the recall and precision scores then we need to designate one of the classes as true and the rest as false

![ee](https://media.licdn.com/mpr/mpr/AAEAAQAAAAAAAAykAAAAJDUzZGVlZGM0LTUyNWMtNDNjZi1hNjkxLTdlZjEzY2VmMmM4OQ.png)

Precision = TP/TP+FP

Recall = TP/TP+FN

Good blog post to refresh your memory on these metrics:http://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/

In [ ]:
#The recall score for setaosa

tp_set = cm_df.loc["actual_setosa", "pred_setosa"]

#Divide tp_set by the sum of values in the actual setosa row

tp_set/float(cm_df.loc["actual_setosa"].sum())

In [ ]:
#The precision score for setosa



#Divide tp_set by the sum of values in the predicted setosa column

tp_set/float(cm_df.pred_setosa.sum())

We got perfect scores but thats boring, let's try this with versicolor

In [ ]:
#The recall score for versicolor

tp_ver = cm_df.loc["actual_versicolor", "pred_versicolor"]

#Divide tp_set by the sum of values in the actual versicolor row

tp_ver/float(cm_df.ix["actual_versicolor"].sum())

In [ ]:
#The precision score for versicolor

#Divide tp_set by the sum of values in the predicted versicolor column

tp_ver/float(cm_df.pred_versicolor.sum())

Multi-class confusion matrix explained:
![s](https://3.bp.blogspot.com/-YpiS7AXxlgs/VEVrZGx5oaI/AAAAAAAAG1c/E8PdwoUamYw/s1600/multi-class-confusionmatrix.png)

[Source](http://text-analytics101.rxnlp.com/2014/10/computing-precision-and-recall-for.html)

## MNIST Dataset

Famous machine learning dataset that is frequently used over and over again in machine learning courses.
http://yann.lecun.com/exdb/mnist/

The hand written digits of the mnist dataset.
![a](https://kuanhoong.files.wordpress.com/2016/01/mnistdigits.gif)

In [ ]:
#Load in dataset from sklearn
from sklearn import datasets

In [ ]:
digits_dict = datasets.load_digits()

In [ ]:
#Data description
digits_dict["DESCR"].split("\n")

In [ ]:
#Load in data

data = digits_dict["data"]

target = digits_dict["target"]

In [ ]:
#Look at data
data

In [ ]:
#Look at target variable
target

In [ ]:
# Print to show there are 1797 images (8 by 8 images for a dimensionality of 64)
print('Image Data Shape' , digits_dict.data.shape)
# Print to show there are 1797 labels (integers from 0–9)
print("Label Data Shape", digits_dict.target.shape)

In [ ]:
#View data of single digit

data[1].reshape(8, 8)

Can you tell what number this data is showing?

Let's visualize the digits

In [ ]:
#Use matplotlib to view the images
plt.figure(figsize=(20,4))
for index, (image, label) in enumerate(zip(data[0:5], target[0:5])):
    plt.subplot(1, 5, index + 1)
    plt.imshow(np.reshape(image, (8,8)), cmap=plt.cm.gray)
    plt.title('Training: %i\n' % label, fontsize = 20)

Now let's use a logistic regression algorithm to model this data

In [ ]:

#Step 1. train test split

X_train, X_test, y_train, y_test = 


#Step 2. Fit DecisionTreeClassifier with max_depth 10 on training data



#Step 3. Make predictions on the X_test using the model




#Step 4. Score predictions



Confusion matrix time

In [ ]:
#Create confusion matrix from predictions and y_test

cm_digits = 

cm_digits

In [ ]:
#Heatmap version

plt.figure(figsize=(10, 8))
sb.heatmap(cm_digits, annot=True, cmap= "coolwarm")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels");

What does this tell us?

In [ ]:
# from sklearn.datasets import fetch_mldata
# mnist = fetch_mldata('MNIST original')

Let's see some of the images we got wrong.

In [ ]:
#Identify indices of wrong predictions
index = 0
false_index = []

In [ ]:
false_preds = y_test != preds

for i, e in enumerate(false_preds):
    if e: 
        false_index.append(i)

In [ ]:
# misclassifiedIndexes

In [ ]:
plt.figure(figsize=(20,4))
for plotIndex, badIndex in enumerate(false_index[0:5]):
    plt.subplot(1, 5, plotIndex + 1)
    plt.imshow(np.reshape(X_test[badIndex], (8,8)), cmap=plt.cm.gray)
    plt.title("Predicted: {}, Actual: {}".format(preds[badIndex], y_test[badIndex]), fontsize = 15)

Class exercise time: 

Which digit did our model do the best at identifying? What digit was the worst?

In [ ]:
#Accuracy score answer 




In [ ]:
#Precision score answer 




In [ ]:
#Recall score answer 




## Imbalanced Class Machine Learning

As if often the case in real-world machine learning, there is a huge imbalance between the distribution of class of the variable we're trying to predicting.

Class False: 98.23%

Class True: 1.77%

Illustrated example
![www](http://www.svds.com/wp-content/uploads/2016/08/messy.png)

The main problem that imbalanced machine learning project present that is they render the accuracy score metric almost irrelevant.

If our null accuracy is 99.5%, then we don't have much room for improvement.

<br>

Imagine a confusion with 1000 TNs, 20 FPs, 15 FNs, and 25 TPs. What would the accuracy, precision, and recall scores be?

In [ ]:
#Confusion matrix form
pd.DataFrame(data=[[1000, 20], [15, 25]], 
             columns=["Predicted_False", "Predicted_True"], 
             index=["Actual_False", "Actual_True"])

In [ ]:
#Assign variables

tn = 1000.
fp = 20.
fn = 15.
tp = 25.

In [ ]:
#Accuracy score

(tp + tn)/(tn + fp + fn + tp)

Pretty good score! Or is it???

In [ ]:
#Whats the null accuracy

(tn + fp)/(tn + fp + fn + tp)

We hardly beat the null accuracy. Now let's calculate precision and recall

In [ ]:
#Precision

tp/(tp + fp)

In [ ]:
#Recall
tp/(tp + fn)

Do we have a good model or not?

Let's move onto the real thing by modeling credit card fraud data

https://www.kaggle.com/dalpozz/creditcardfraud

In [ ]:
path = "../../data/fraud.csv"

fraud = pd.read_csv(path, index_col=[0])

fraud.drop("Time", axis = 1, inplace = True)
fraud.head()

The V features are principal components which we'll learn about next Thursday, in the meantime think of them as hidden features.

Let'see how imbalanced the classes are

In [ ]:
#Value counts without normalize


In [ ]:
#Value counts with normalize


That is pretty imbalanced would you say?

In [ ]:
#Quick EDA to find relationship amount and fraud status

fraud.groupby("Class").mean()["Amount"]

What does this say about the relationship between amount of the transaction and fraudelent status?

Before we get into modeling, what metric should we try to minimize False Negatives or False Positives? Why?

Let's do some modeling.

Train a logistic regression model and evaluate it on the testing dataset using accuracy, recall, and precision scores

In [ ]:
#Assign variables

X = fraud.drop("Class", axis = 1)

y = fraud.Class

In [ ]:
#Make a train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 25)

In [ ]:
#Fit logistic regression model on training data




In [ ]:
#Null accuracy of testing data



In [ ]:
#Evaluate it on testing set using accuracy score

preds = 


In [ ]:
#Evaluate it on testing set using precision score



In [ ]:
#Evaluate it on testing set using recall score



In [ ]:
#Confusion matrix



What do these metrics tell us about our dataset?

Cross validation time

In [ ]:
#Cross validate using precision score



In [ ]:
#Cross validate using precision score



In [ ]:
#Cross validate using roc_auc score



Make a roc curve

In [ ]:
#Derive probabilities of class 1 from the test set


#Pass in the test_probs variable and the true test labels aka y_test in the roc_curve function


#Outputs the fpr, tpr, for varying thresholds

In [ ]:
#Plot ROC_curve again but this time annotate the curve with the threshold value
plt.figure(figsize=(15,11))
plt.plot(fpr, tpr, linewidth=12, alpha = .6)
plt.plot([0,1], [0,1], "--", alpha=.6)
for label, x, y in zip(thres[::10], fpr[::10], tpr[::10]):
    plt.annotate("{0:.2f}".format(label), xy=(x, y ), size = 15)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

How do you interpret this chart?

### Imbalanced class techniques

![eop](https://chrisalbon.com/images/machine_learning_flashcards/Downsampling_print.png)
Source: Chris Albon

Let's go ahead apply down sampling to our training dataset

In [ ]:
#How many true class are there in the training dataset?

fraud.Class.value_counts()

In [ ]:
#Assign number of fraud class in training data to variable
N = fraud.Class.value_counts()[1]
N

In [ ]:
#Import resample function from sklearn
from sklearn.utils import resample

fraud_maj = fraud[fraud.Class==0]
fraud_min = fraud[fraud.Class==1]
 
# Downsample majority class
fraud_majority_downsampled = resample(fraud_maj, 
                                 replace=False,     # Do not sample with replacement
                                 n_samples=N,    # to match minority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
fraud_ds = pd.concat([fraud_majority_downsampled, fraud_min])
 
# Display new class counts
fraud_ds.Class.value_counts()

Perfectly balanced classes. Let's use cross validate to see how well our model does.

Use accuracy, recall, precision, and roc_auc metrics

In [ ]:
X_ds = fraud_ds.drop("Class", axis = 1)
y_ds = fraud_ds.Class

cross_val_score(LogisticRegression(), X_ds, y_ds, cv = 5, scoring="accuracy").mean()

In [ ]:
#Precision
cross_val_score(LogisticRegression(), X_ds, y_ds, cv = 5, scoring="precision").mean()

In [ ]:
#Precision
cross_val_score(LogisticRegression(), X_ds, y_ds, cv = 5, scoring="recall").mean()

In [ ]:
#Precision
cross_val_score(LogisticRegression(), X_ds, y_ds, cv = 5, scoring="roc_auc").mean()

What does this tell us about our model and our data?

![aw](https://chrisalbon.com/images/machine_learning_flashcards/Upsampling_print.png)

In [ ]:

#Number non fraud observations
N = fraud.Class.value_counts()[0]

# Downsample majority class
fraud_minority_upsampled = resample(fraud_min, 
                                 replace=True,     # Do not sample with replacement
                                 n_samples=N,    # to match minority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
fraud_us = pd.concat([fraud_minority_upsampled, fraud_maj])
 
# Display new class counts
fraud_us.Class.value_counts()

Cross validation again

In [ ]:
#Assign X and y
X_us = fraud_us.drop("Class", axis = 1)
y_us = fraud_us.Class

#Accuracy score
cross_val_score(LogisticRegression(), X_us, y_us, cv = 5, scoring="accuracy").mean()

In [ ]:
#Precision score
cross_val_score(LogisticRegression(), X_us, y_us, cv = 5, scoring="precision").mean()

In [ ]:
#Recall
cross_val_score(LogisticRegression(), X_us, y_us, cv = 5, scoring="recall").mean()

In [ ]:
#Roc auc score
cross_val_score(LogisticRegression(), X_us, y_us, cv = 5, scoring="roc_auc").mean()

With both techniques, our interpretation of the accuracy score is more meaningful.

![wee](https://svds.com/wp-content/uploads/2016/08/ImbalancedClasses_fig5.jpg)

However, there is an issue here and that is can a model trained on balanced data work well with imbalanced data? Let's find out!


We're going to train a logistic regression model on a downsampled training dataset and then apply it to an imbalanced testing dataset.

In [ ]:
#reassign variables

X = fraud.drop("Class", axis = 1)

y = fraud.Class

In [ ]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.4, 
                                                    random_state = 23)

In [ ]:
# X_train

Downsample data

In [ ]:
#Combine the two training datasets
train = X_train.copy()
train["Class"] = y_train

In [ ]:
#Class count
train.Class.value_counts()

In [ ]:
N = train.Class.value_counts()[1]

fraud_maj = train[train.Class==0]
fraud_min = train[train.Class==1]
 
# Downsample majority class
fraud_majority_downsampled = resample(fraud_maj, 
                                 replace=False,     # Do not sample with replacement
                                 n_samples=N,    # to match minority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
fraud_ds = pd.concat([fraud_majority_downsampled, fraud_min])
 
# Display new class counts
fraud_ds.Class.value_counts()


Train Logistic Regression on downsampled data and evaluate it on testing data

In [ ]:
#Assign X and y
fraud_ds_X = fraud_ds.drop("Class", axis = 1)
fraud_ds_y = fraud_ds.Class
#Intialize
lr = LogisticRegression()
lr.fit(fraud_ds_X, fraud_ds_y)

In [ ]:
#Null accuracy 

y_test.value_counts(normalize=True)

In [ ]:
#Evaluate on testing dataset

preds = lr.predict(X_test)

accuracy_score(y_test, preds)

In [ ]:
#precision

precision_score(y_test, preds)

In [ ]:
#recall

recall_score(y_test, preds)

In [ ]:
#Confusion matrix
confusion_matrix(y_test, preds)

What's your interpretation now?

<br>
Let's the upsampling technique to see if that produces a better model.

In [ ]:
N = train.Class.value_counts()[0]

fraud_maj = train[train.Class==0]
fraud_min = train[train.Class==1]
 
# Downsample majority class
fraud_minority_upsampled = resample(fraud_min, 
                                 replace=True,     # Do not sample with replacement
                                 n_samples=N,    # to match minority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
fraud_us = pd.concat([fraud_minority_upsampled, fraud_maj])
 
# Display new class counts
fraud_us.Class.value_counts()

In [ ]:
#Assign X and y
fraud_us_X = fraud_ds.drop("Class", axis = 1)
fraud_us_y = fraud_ds.Class
#Intialize
lr = LogisticRegression()
lr.fit(fraud_us_X, fraud_us_y)

In [ ]:
#Evaluate on testing dataset

preds = lr.predict(X_test)

accuracy_score(y_test, preds)

In [ ]:
#precision

precision_score(y_test, preds)

In [ ]:
#recall

recall_score(y_test, preds)

In [ ]:
#Confusion matrix
confusion_matrix(y_test, preds)

What do we make of these results??

Here's the good news. We can set a class_weight setting in our models to be assigned to "balanced".

From sklearn:

"The 'balanced' mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))."



We'll cross validate with Logistic Regression, Decision Trees, and RandomForest models with the class_weight parameter set to "balanced."

But first let's calculate those weights

In [ ]:
#Class 0 weight
(y.shape[0])/float((2*y.value_counts()[0]))

In [ ]:
#Class 1 weight
(y.shape[0])/float((2*y.value_counts()[1]))

In [ ]:
y.value_counts()

In [ ]:
#multiple class counts by weights
64315*0.5038249242011972

In [ ]:

65*492

In [ ]:
#Logistic regression 

cross_val_score(LogisticRegression(class_weight="balanced"), 
                X, y, cv = 5, scoring="accuracy").mean()

In [ ]:
#Decision Tree with max depth = 12

cross_val_score(DecisionTreeClassifier(class_weight="balanced", max_depth=14), 
                X, y, cv = 5, scoring="accuracy").mean()

In [ ]:
#Random Forest with n_estimators = 40

cross_val_score(RandomForestClassifier(class_weight="balanced", n_estimators=40), 
                X, y, cv = 5, scoring="accuracy").mean()

# Resources

MNIST:

- https://github.com/grfiv/MNIST/blob/master/knn/knn.ipynb
- https://github.com/monsta-hd/ml-mnist
- https://www.youtube.com/watch?v=aZsZrkIgan0
- http://joshmontague.com/posts/2016/mnist-scikit-learn/

Multi-class:

- https://gallery.cortanaintelligence.com/Competition/Tutorial-Competition-Iris-Multiclass-Classification-2
-https://www.youtube.com/watch?v=6kzvrq-MIO0


Imbalanced classes:

- https://towardsdatascience.com/what-metrics-should-we-use-on-imbalanced-data-set-precision-recall-roc-e2e79252aeba
- https://svds.com/learning-imbalanced-classes/
- https://www.youtube.com/watch?v=X9MZtvvQDR4
- https://elitedatascience.com/imbalanced-classes
- https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/

Grid Search and Pipelines

- https://chrisalbon.com/machine-learning/cross_validation_parameter_tuning_grid_search.html
- https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/
- https://www.youtube.com/watch?v=Gol_qOgRqfA
- https://chrisalbon.com/machine-learning/pipelines_with_parameter_optimization.html
- https://chrisalbon.com/machine-learning/hyperparameter_tuning_using_random_search.html
- https://machinelearningmastery.com/automate-machine-learning-workflows-pipelines-python-scikit-learn/
- https://www.civisanalytics.com/blog/workflows-in-python-using-pipeline-and-gridsearchcv-for-more-compact-and-comprehensive-code/

## Bonus: Gridsearch with Regression

In [ ]:
#Load in boston dataset
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
boston = load_boston()
df = pd.DataFrame(boston["data"])
df.columns = boston["feature_names"]
df["MEDV"] = boston["target"]
df.head()

In [ ]:
#Assign X and y

X = df.drop("MEDV", axis =1)
y = df.MEDV

In [ ]:
#Use a Pipeline Class instead of function to establish pipeline
pipe_poly = Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                           ('linearregression', LinearRegression())])                           

In [ ]:
#Select a few features from X
XX = X[["RM", "DIS", "NOX", "CRIM"]].copy()

In [ ]:
#Intialize range values for poly
poly_range = [1, 2, 3, 4, 5, 6, 7]

#Intialize grid dictionary
param_grid_poly = {}

#Input grid values
param_grid_poly["polynomialfeatures__degree"] = poly_range

#Establish the grid
poly_grid = GridSearchCV(pipe_poly, 
                         param_grid = param_grid_poly, cv=5, 
                         scoring='neg_mean_squared_error')

In [ ]:
poly_grid.fit(XX, y)

In [ ]:
pipe_poly = Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                            ('ridgeregression', Ridge())]) 

param_grid_ridge = {'polynomialfeatures__degree': [1, 2, 3, 4, 5],
              'ridgeregression__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

grid_ridge = RandomizedSearchCV(pipe_poly, param_distributions=param_grid_ridge, 
                                n_iter = 5 , cv = 5, scoring='neg_mean_squared_error')
grid_ridge.fit(XX, y)

In [ ]:

print (grid_ridge.best_params_, grid_ridge.best_score_)